In [1]:
!pip install ultralytics opencv-python numpy pycocotools

In [2]:
!pip install segmentation-models-pytorch torch torchvision albumentations
!pip install geopandas rasterio scikit-learn

In [11]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()


In [12]:
import os
import json
import shutil
from pathlib import Path
import numpy as np
from PIL import Image
from rasterio import features
import rasterio
from shapely.geometry import Polygon

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import segmentation_models_pytorch as smp
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.cuda.amp import autocast, GradScaler
from skimage import measure
from typing import List, Tuple, Union

# --- CONFIGURATION ---
RAW_DATA_DIR = Path("data/raw")
PROCESSED_DATA_DIR = Path("data/processed_unet")
SOLAFUNE_JSON_PATH = RAW_DATA_DIR / "train_annotations.json"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

ENCODER = 'resnet18'
ENCODER_WEIGHTS = 'imagenet'
EPOCHS = 50
BATCH_SIZE = 2
CLASSES = ["individual_tree", "group_of_trees"]  # Multi-class
NUM_CLASSES = len(CLASSES)

# ------------------------
# 1. DATA PREPARATION
# ------------------------
def parse_solafune_json_for_polygons(json_path):
    annotations = {}
    with open(json_path, 'r') as f:
        data = json.load(f)
    for image_data in data.get('images', []):
        filename = image_data['file_name']
        width, height = image_data['width'], image_data['height']
        annotations[filename] = {'size': (width, height), 'polygons': []}
        for ann in image_data.get('annotations', []):
            flat_points = ann['segmentation']
            points = list(zip(flat_points[::2], flat_points[1::2]))
            class_name = ann.get('class', 'group_of_trees')
            if len(points) >= 3:
                annotations[filename]['polygons'].append({'polygon': Polygon(points), 'class': class_name})
    return annotations

def create_and_save_masks(all_annotations, raw_image_dir, output_dir):
    print("Creating and saving multi-channel masks...")
    images_out_dir = output_dir / "images"
    masks_out_dir = output_dir / "masks"
    images_out_dir.mkdir(parents=True, exist_ok=True)
    masks_out_dir.mkdir(parents=True, exist_ok=True)

    for filename, data in all_annotations.items():
        shutil.copy(raw_image_dir / filename, images_out_dir / filename)
        width, height = data['size']

        mask_array = np.zeros((NUM_CLASSES, height, width), dtype=np.uint8)
        for ann in data['polygons']:
            poly, cls_name = ann['polygon'], ann['class']
            if cls_name in CLASSES:
                cls_idx = CLASSES.index(cls_name)
                mask_array[cls_idx] = np.maximum(mask_array[cls_idx],
                                                 features.rasterize([poly], out_shape=(height, width),
                                                                    transform=rasterio.Affine.identity(),
                                                                    fill=0, all_touched=True, dtype=np.uint8))
        # Save masks as multi-channel npy
        np.save(masks_out_dir / f"{Path(filename).stem}.npy", mask_array)

def prepare_data():
    raw_image_dir = RAW_DATA_DIR / "train_images"
    all_annotations = parse_solafune_json_for_polygons(SOLAFUNE_JSON_PATH)
    create_and_save_masks(all_annotations, raw_image_dir, PROCESSED_DATA_DIR)
    print(f"Images saved to: {PROCESSED_DATA_DIR / 'images'}")
    print(f"Masks saved to: {PROCESSED_DATA_DIR / 'masks'}")

# ------------------------
# 2. DATASET
# ------------------------
class TreeCanopyDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        mask = np.load(self.mask_paths[idx])  # shape: [num_classes, H, W]

        if self.transform:
            transformed = self.transform(image=image, mask=mask.transpose(1,2,0))
            image = transformed['image']
            mask = transformed['mask'].permute(2,0,1).float()  # back to [C,H,W]

        return image, mask

# ------------------------
# 3. AUGMENTATIONS
# ------------------------
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Transpose(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
    ToTensorV2()
])

# ------------------------
# 4. TRAINING
# ------------------------
def train_model():
    data_dir = PROCESSED_DATA_DIR
    all_image_paths = sorted(list((data_dir / "images").glob("*.*")))
    all_mask_paths = sorted(list((data_dir / "masks").glob("*.npy")))

    train_imgs, val_imgs, train_msks, val_msks = train_test_split(
        all_image_paths, all_mask_paths, test_size=0.2, random_state=42
    )

    train_dataset = TreeCanopyDataset(train_imgs, train_msks, transform=train_transform)
    val_dataset = TreeCanopyDataset(val_imgs, val_msks, transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    model = smp.Unet(encoder_name=ENCODER, encoder_weights=ENCODER_WEIGHTS,
                     in_channels=3, classes=NUM_CLASSES, activation='softmax').to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scaler = GradScaler()

    best_val_loss = float('inf')
    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0
        for images, masks in train_loader:
            images, masks = images.to(DEVICE), masks.to(DEVICE)
            optimizer.zero_grad()
            with autocast():
                outputs = model(images)
                # targets for CELoss: shape [B,H,W], values: 0..C-1
                targets = masks.argmax(dim=1).long()
                loss = criterion(outputs, targets)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            running_loss += loss.item()

        # validation
        model.eval()
        val_loss = 0
        with torch.no_grad(), autocast():
            for images, masks in val_loader:
                images, masks = images.to(DEVICE), masks.to(DEVICE)
                outputs = model(images)
                targets = masks.argmax(dim=1).long()
                val_loss += criterion(outputs, targets).item()
        val_loss /= len(val_loader)
        print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}")
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_unet_model_multiclass.pth")
            print("   -> Best model saved!")

    return model

# ------------------------
# 5. RUN EVERYTHING
# ------------------------
if __name__ == "__main__":
    prepare_data()
    model = train_model()
    evaluate_model(model, Path("data/test/evaluation_images"))


Creating and saving multi-channel masks...
Images saved to: data\processed_unet\images
Masks saved to: data\processed_unet\masks


C:\Users\hmanasi1\.conda\envs\adml_py311\Lib\site-packages\segmentation_models_pytorch\base\modules.py:182: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.activation(x)


Epoch 1/50, Train Loss: 0.5719, Val Loss: 0.4955
   -> Best model saved!
Epoch 2/50, Train Loss: 0.4638, Val Loss: 0.4373
   -> Best model saved!
Epoch 3/50, Train Loss: 0.4304, Val Loss: 0.4223
   -> Best model saved!
Epoch 4/50, Train Loss: 0.4158, Val Loss: 0.3986
   -> Best model saved!
Epoch 5/50, Train Loss: 0.4073, Val Loss: 0.3891
   -> Best model saved!
Epoch 6/50, Train Loss: 0.3959, Val Loss: 0.3829
   -> Best model saved!
Epoch 7/50, Train Loss: 0.3885, Val Loss: 0.3899
Epoch 8/50, Train Loss: 0.3862, Val Loss: 0.3830
Epoch 9/50, Train Loss: 0.3850, Val Loss: 0.3775
   -> Best model saved!
Epoch 10/50, Train Loss: 0.3863, Val Loss: 0.3789
Epoch 11/50, Train Loss: 0.3802, Val Loss: 0.3811
Epoch 12/50, Train Loss: 0.3789, Val Loss: 0.3863
Epoch 13/50, Train Loss: 0.3795, Val Loss: 0.3717
   -> Best model saved!
Epoch 14/50, Train Loss: 0.3739, Val Loss: 0.3720
Epoch 15/50, Train Loss: 0.3713, Val Loss: 0.3873
Epoch 16/50, Train Loss: 0.3772, Val Loss: 0.3748
Epoch 17/50, Trai

KeyboardInterrupt: 

In [16]:
import json
from pathlib import Path
import numpy as np
from PIL import Image
from shapely.geometry import Polygon, box
import torch
from tqdm import tqdm  # progress bar
import cv2

# --- CONFIG ---
EVAL_DIR = Path("data/test/evaluation_images")  # Folder with evaluation images
OUTPUT_JSON_PATH = EVAL_DIR / "jsons/unet_evaluation.json"  # Single JSON file
CONFIDENCE_METHOD = 'mean'  # How to assign confidence score for each polygon
CLASSES = ["individual_tree", "group_of_trees"]  # Must match your model
DEBUG = True  # Set False to disable prints

def mask_to_polygons_with_scores(mask, confidence_map, min_area=10):
    """
    Convert binary mask to list of (Polygon, confidence_score)
    Each polygon is simplified to 4 points (bounding box)
    """
    polygons_with_scores = []
    mask_uint8 = (mask > 0.5).astype(np.uint8) * 255
    contours, _ = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for cnt in contours:
        if cv2.contourArea(cnt) < min_area:
            continue
        
        # Compute minimum bounding rectangle
        rect = cv2.minAreaRect(cnt)  # ((cx,cy),(w,h),angle)
        box_pts = cv2.boxPoints(rect)  # 4 points, float32
        poly = Polygon(box_pts)
        
        # Confidence score: mean of mask under this contour
        mask_indices = mask_uint8 > 0
        score = confidence_map[mask_indices].mean() if CONFIDENCE_METHOD=='mean' else 1.0
        polygons_with_scores.append((poly, float(score)))
    
    return polygons_with_scores

# --- Run inference ---
model.eval()
all_images_json = []

print("Starting evaluation on images...")
for img_path in tqdm(sorted(EVAL_DIR.glob("*.tif")), desc="Images"):
    image = np.array(Image.open(img_path).convert("RGB"))
    image_tensor = train_transform(image=image)['image'].unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        output = model(image_tensor)  # [B, C, H, W]
    output_np = output.squeeze(0).cpu().numpy()  # [C, H, W]

    annotations = []

    for cls_idx, class_name in enumerate(CLASSES):
        class_mask = (output_np[cls_idx] > 0.5).astype(np.uint8)
        pred_polys = mask_to_polygons_with_scores(class_mask, output_np[cls_idx])

        if DEBUG:
            print(f"{img_path.name} - Class '{class_name}' - {len(pred_polys)} polygons detected")

        for poly, score in pred_polys:
            coords = np.array(poly.exterior.coords).flatten().tolist()[:8]  # only first 4 points
            annotations.append({
                "class": class_name,
                "confidence_score": float(score),
                "segmentation": [float(c) for c in coords]
            })

    all_images_json.append({
        "file_name": img_path.name,
        "width": image.shape[1],
        "height": image.shape[0],
        "cm_resolution": 10,  # adjust if known
        "scene_type": "unknown",
        "annotations": annotations
    })

# Save all images in a single JSON file
OUTPUT_JSON_PATH.parent.mkdir(exist_ok=True, parents=True)
with open(OUTPUT_JSON_PATH, "w") as f:
    json.dump({"images": all_images_json}, f, indent=4)

print(f"\nAll evaluation predictions saved to: {OUTPUT_JSON_PATH}")


Starting evaluation on images...


Images:   1%|▉                                                                         | 2/150 [00:00<00:42,  3.50it/s]

10cm_evaluation_1.tif - Class 'individual_tree' - 1 polygons detected
10cm_evaluation_1.tif - Class 'group_of_trees' - 3 polygons detected
10cm_evaluation_10.tif - Class 'individual_tree' - 2 polygons detected
10cm_evaluation_10.tif - Class 'group_of_trees' - 15 polygons detected


Images:   2%|█▍                                                                        | 3/150 [00:00<00:30,  4.77it/s]

10cm_evaluation_11.tif - Class 'individual_tree' - 1 polygons detected
10cm_evaluation_11.tif - Class 'group_of_trees' - 16 polygons detected
10cm_evaluation_12.tif - Class 'individual_tree' - 2 polygons detected


...

Images:  99%|███████████████████████████████████████████████████████████████████████ | 148/150 [00:23<00:00,  6.88it/s]

80cm_evaluation_147.tif - Class 'individual_tree' - 1 polygons detected
80cm_evaluation_147.tif - Class 'group_of_trees' - 42 polygons detected
80cm_evaluation_148.tif - Class 'individual_tree' - 1 polygons detected
80cm_evaluation_148.tif - Class 'group_of_trees' - 3 polygons detected


Images: 100%|████████████████████████████████████████████████████████████████████████| 150/150 [00:23<00:00,  6.35it/s]

80cm_evaluation_149.tif - Class 'individual_tree' - 1 polygons detected
80cm_evaluation_149.tif - Class 'group_of_trees' - 23 polygons detected
80cm_evaluation_150.tif - Class 'individual_tree' - 1 polygons detected
80cm_evaluation_150.tif - Class 'group_of_trees' - 4 polygons detected



All evaluation predictions saved to: data\test\evaluation_images\jsons\unet_evaluation.json


In [18]:
# Notebook: unet_train_image_prediction.ipynb
import json
from pathlib import Path
import numpy as np
from PIL import Image
from shapely.geometry import Polygon
import torch
import cv2
import segmentation_models_pytorch as smp
from albumentations.pytorch import ToTensorV2
import albumentations as A

# --- CONFIG ---
TRAIN_IMAGE = Path("data/raw/train_images/10cm_train_3.tif")
OUTPUT_JSON_PATH = Path("data/raw/train_images/jsons/unet_train_image_predictions.json")
CONFIDENCE_THRESHOLD = 0.5
CLASSES = ["individual_tree", "group_of_trees"]
UNET_MODEL_PATH = "best_unet_model_multiclass.pth"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ------------------------------
# Helper: Convert mask to polygons
# ------------------------------
def mask_to_polygons(mask, confidence_map=None, min_area=10):
    polygons_with_scores = []
    mask_uint8 = (mask > 0.5).astype(np.uint8) * 255
    contours, _ = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        if cv2.contourArea(cnt) >= min_area:
            poly = Polygon(cnt[:,0,:])
            score = confidence_map[mask_uint8>0].mean() if confidence_map is not None else 1.0
            polygons_with_scores.append((poly, float(score)))
    return polygons_with_scores

# ------------------------------
# Load UNet model
# ------------------------------
transform = A.Compose([
    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
    ToTensorV2()
])

model_unet = smp.Unet(
    encoder_name='resnet18',
    encoder_weights=None,
    in_channels=3,
    classes=len(CLASSES),
    activation='softmax'
).to(DEVICE)
model_unet.load_state_dict(torch.load(UNET_MODEL_PATH, map_location=DEVICE))
model_unet.eval()

# ------------------------------
# Run prediction
# ------------------------------
image_np = np.array(Image.open(TRAIN_IMAGE).convert("RGB"))
image_tensor = transform(image=image_np)['image'].unsqueeze(0).to(DEVICE)

with torch.no_grad():
    output = model_unet(image_tensor)
output_np = output.squeeze(0).cpu().numpy()

annotations_unet = []
for cls_idx, class_name in enumerate(CLASSES):
    mask = (output_np[cls_idx] > CONFIDENCE_THRESHOLD).astype(np.uint8)
    polys = mask_to_polygons(mask, confidence_map=output_np[cls_idx])
    for poly, score in polys:
        coords = np.array(poly.exterior.coords).flatten().tolist()
        annotations_unet.append({
            "class": class_name,
            "confidence_score": float(score),
            "segmentation": [float(c) for c in coords]
        })

# ------------------------------
# Save JSON
# ------------------------------
output_json = {
    "file_name": TRAIN_IMAGE.name,
    "width": image_np.shape[1],
    "height": image_np.shape[0],
    "cm_resolution": 10,
    "scene_type": "unknown",
    "annotations_unet": annotations_unet
}

OUTPUT_JSON_PATH.parent.mkdir(parents=True, exist_ok=True)
with open(OUTPUT_JSON_PATH, 'w') as f:
    json.dump(output_json, f, indent=4)

print(f"UNet prediction saved to: {OUTPUT_JSON_PATH}")
print(f"UNet polygons: {len(annotations_unet)}")


UNet prediction saved to: data\raw\train_images\jsons\unet_train_image_predictions.json
UNet polygons: 13
